In [109]:
from utils import extract_scaling_metadata
import os
from tqdm import tqdm
from pathlib import Path
import czifile
import napari
import numpy as np
import skimage
from skimage import measure, exposure
from skimage.measure import regionprops_table
import pandas as pd
import pyclesperanto_prototype as cle
import plotly.express as px
from cellpose import models, utils

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (2 refs)>

In [110]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    # Remove Control and Isotype stainings from the analysis
    if "Control" and "Isotype" not in file_path.stem:
        images.append(str(file_path))
    
len(images)

117

In [111]:
image = images[1]

# Read path storing raw image and extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file and remove singleton dimensions
img = czifile.imread(image)
img = img.squeeze()

# Extract experiment, mouse, treatment and replica ids
experiment_id = filename.split(" ")[0]
mouse_id = filename.split(" ")[1]
treatment_id = filename.split(" ")[2]
replica_id = filename.split(" ")[-1]

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Extract the stack containing the UEA-1 (0), YAP (1), nuclei (2) and BCAT (3) channels.
uea1_stack = img[0, :, ::slicing_factor, ::slicing_factor]
yap_stack = img[1, :, ::slicing_factor, ::slicing_factor]
nuclei_stack = img[2, :, ::slicing_factor, ::slicing_factor]
bcat_stack = img[3, :, ::slicing_factor, ::slicing_factor]

In [112]:

# Extract x,y,z scaling from .czi file metadata in order to make data isotropic
scaling_x_um, scaling_y_um, scaling_z_um = extract_scaling_metadata(file_path)

# Resample nuclei stack for Cellpose prediction
resampled_nuclei = cle.scale(nuclei_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
background_subtracted = cle.top_hat_box(resampled_nuclei, radius_x=5, radius_y=5, radius_z=5)

# Pull OCL arrays from GPU memory and transform into numpy arrays to be fed into Cellpose
processed_nuclei = cle.pull(resampled_nuclei)
del background_subtracted

# Apply Contrast Stretching to improve Cellpose detection of overly bright nuclei
p2, p98 = np.percentile(processed_nuclei, (2, 98)) 
nuclei_rescaled = exposure.rescale_intensity(processed_nuclei, in_range=(p2, p98))

# Initialize the Cellpose model
model = models.Cellpose(gpu=True, model_type='nuclei')

# Run Cellpose
masks, flows, styles, diams = model.eval(nuclei_rescaled, diameter=10, channels=[0, 0], do_3D=True)


In [113]:
# Resample stacks for visualization
resampled_yap = cle.scale(yap_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
resampled_bcat = cle.scale(bcat_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)

In [114]:
# Initialize Napari viewer to display segmentation results
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(processed_nuclei)
viewer.add_image(resampled_yap)
viewer.add_image(resampled_bcat)
viewer.add_labels(masks)


<Labels layer 'masks' at 0x206d4ef9b80>

In [115]:
# Extract regionprops
props = regionprops_table(label_image=masks, intensity_image=processed_nuclei, properties=["label", "intensity_mean", "equivalent_diameter_area", "area"])

df = pd.DataFrame(props)

df[df['label']==153]

,label,intensity_mean,equivalent_diameter_area,area
152,153,69.734375,4.962804,64.0


In [116]:
# Create histogram
fig = px.histogram(df, x="area", nbins=300, labels={"area": "Area"}, title="Histogram of Areas of Different Labels")

# Show plot
fig.show()

In [117]:
# Exclude nuclei labels based on size in pixels

# Create a destination array with the same shape as masks
destination = cle.create_like(masks)

# Remove labels smaller than 90 pixels (artifacts)
filtered_nuclei_gpu = cle.exclude_small_labels(masks, destination, 90)

# Remove labels larger than 500 pixels (merged nuclei)
filtered_nuclei_gpu = cle.exclude_large_labels(filtered_nuclei_gpu, destination, 500)

# Pull filtered_nuclei from GPU
filtered_nuclei = cle.pull(filtered_nuclei_gpu).astype(np.uint16)

# Dilate labels
dilated_nuclei_gpu = cle.dilate_labels(filtered_nuclei, radius=2)
dilated_nuclei = cle.pull(dilated_nuclei_gpu)


In [118]:
# Create a copy of dilated_nuclei to modify
cytoplasm = dilated_nuclei.copy()

# Get unique labels (excluding 0 which is background)
unique_labels = np.unique(filtered_nuclei)
unique_labels = unique_labels[unique_labels != 0]

# Iterate over each label and remove the corresponding pixels from dilated_nuclei
for label in unique_labels:
    # Create a mask for the current label in filtered_nuclei
    mask = (filtered_nuclei == label)
    
    # Set corresponding pixels in resulting_nuclei to zero
    cytoplasm[mask] = 0

viewer.add_labels(cytoplasm)

<Labels layer 'cytoplasm' at 0x206d8c0a520>